In [ ]:
# 匯入套件
import requests as req 
from bs4 import BeautifulSoup as bs
import re #正則表達,因只想抓取中文內容
import os
from fake_useragent import UserAgent
ua = UserAgent(cache=True)

In [ ]:
folderPath = 'gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath) #確定放置抓取內容資料夾,不存在則自動創建

url = 'https://www.gutenberg.org/browse/languages/zh' #目標網址

my_headers = {
    'user-agent':ua.random
} #user-agent自動random

res = req.get(url, headers = my_headers) #創建requests

soup = bs(res.text, 'lxml') #beautifulsoup解析網站

list1 = [] #設一個空list放網址

prefix = 'https://www.gutenberg.org/'

In [ ]:
for a in soup.select('li.pgdbetext > a'):
    list1.append(prefix + a['href'])
    
for index, link in enumerate(list1):
    res1 = req.get(link, headers = my_headers)
    soup1 = bs(res1.text, 'lxml')
    try:
        b = soup1.select_one('a[href$="html.noimages"]')
        res2 = req.get(prefix + b['href'])
    except:
        continue
    res2.encoding = 'UTF-8'
    soup2 = bs(res2.text, 'lxml')
    txt = soup2.select_one('body') 
    regex01 = r'[\u4E00-\u9FFF]+\S'
    match01 = re.findall(regex01, txt.get_text())
    str01 = "".join(match01) 
    #抓取文章內文設為字串str01

    regex02 = r'[\u4E00-\u9FFF]+'
    txt2 = soup1.select_one('h1')
    match02 = re.findall(regex02, txt2.get_text())
    str02 = "".join(match02)
    #抓取文章標題設為字串str02

    with open(f'./{folderPath}/{str02}.txt' , 'w' , encoding='UTF-8') as f:
        f.write(str01)
    #以標題為檔案名稱儲存文章至資料夾